# Elementos do dataset

Antes mesmo de falar do projeto, ao longo do desenvolvimento, vamos explicitando qual o objetivo de cada versão de trabalho do arquivo principal.

O objetivo desta AvoidMort_v5_BR_CID_UF é criar um dataset de mortalidades gerais para servir de comparação para a mortalidade evitável.

O dataset final deverá ter as seguintes características:

* setting: Brasil [poderemos encontrar um sufixo a Brasil que permita a sua subclassificação e carga deste dataset junto a outros]
* dimension: UF
* subgroup: sexo & raça
* estimate: mortalidade por CIDs evitáveis e município
* setting-average: mortalidade por CIDs evitáveis por Brasil

# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




### Instalação e importação de bibliotecas

In [1]:
# importar biblioteca básicas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown
import openpyxl
from tabulate import tabulate

In [2]:
# Para organizar resultados, gerar prints em amarelo

def print_y(text):
  """Imprime o texto fornecido na cor amarela.

  Args:
    text: O texto a ser impresso.
  """
  print(f"\033[33m ==> {text}\033[0m")

# Exemplo de uso:
print_y("Este texto será impresso em amarelo!")
print("Este em normal!")

 ==> Este texto será impresso em amarelo!
Este em normal!


# Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [ ]:
# # Download de tabela tratada de CIDs evitáveis de acordo com documentos da OCDE
# url = 'https://docs.google.com/spreadsheets/d/1X8AH0zs4PNc_bUpZ3tBvFw2HGpgb9CjShOkwXQ0so_I/edit?usp=sharing'
# # Convert the Google Sheets URL to a CSV export URL
# csv_url = url.replace('/edit?usp=sharing', '/export?format=csv')

# # Read the CSV data into a pandas DataFrame
# ocde_evit = pd.read_csv(csv_url)

# # Now you can work with the DataFrame 'df'
# ocde_evit.head()

In [3]:
ocde_evit_temp = pd.read_csv('files_in_geral/cid10_evit_ocde.csv')

# mostrar dados gerais da tabela após transformações
tab = ocde_evit_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))

Registro da tabela : 346
+----+---------------------+-----------------------+--------------+---------------------+------------------------------------+---------------------------------------------------+-----------------+-----------+
|    | avoid_flag          | prevent_flag          |   treat_flag | Group               | Causes of deaths                   | Rationale for inclusion                           | Range           | cid_pai   |
|----+---------------------+-----------------------+--------------+---------------------+------------------------------------+---------------------------------------------------+-----------------+-----------|
|  0 | Avoidable mortality | Preventable mortality |          nan | Infectious diseases | Intestinal diseases                | Most of these infections can be prevented through | (A00-A09)       | A0        |
|    |                     |                       |              |                     |                                    | prevention m

In [4]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_in_geral/dobr_10_19a22_u75.parquet')
causas_unicas = dobr_evit_merged_temp['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11895 entries, 0 to 11894
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  11895 non-null  object
dtypes: object(1)
memory usage: 93.1+ KB


##########         Etapa diferenciada da versão          ##########

Utilizar 'Causes of deaths' de CID evitáveis e as demais como "Causa não evitável"

In [5]:
# Fazer merge da lista de causas_unicas com a lista de causas evitáveis

df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit_temp
column_b = 'cid_pai'
column_return = 'Causes of deaths'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['causes_deaths'] = df_a.apply(verificar_correspondencia, axis=1)

print(tabulate(df_a.head()))

-  ----  -------------------
0  A049  Intestinal diseases
1  A09   Intestinal diseases
2  A162  Tuberculosis
3  A169  Tuberculosis
4  A199  Tuberculosis
-  ----  -------------------


In [6]:
# Merge dobr_merged_pop_clean and df_a by CAUSABAS
dobr_evit_merged_temp01 = pd.merge(dobr_evit_merged_temp, df_a, on='CAUSABAS', how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4427475
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Intestinal diseases |
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+


In [7]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp01.groupby('causes_deaths').count()

,CAUSABAS,CODMUNRES,SEXO,RACACOR,OBITO_SOMA,ANO_OBITO
causes_deaths,,,,,,
Abdominal hernia,5886,5886,5886,5886,5886,5886
Abscess of lung and mediastinum pyothorax,2512,2512,2512,2512,2512,2512
Accidental Injuries,101006,101006,101006,101006,101006,101006
Acute lower respiratory infections,3261,3261,3261,3261,3261,3261
Acute pancreatitis,11836,11836,11836,11836,11836,11836
...,...,...,...,...,...,...
Varicella,270,270,270,270,270,270
Venous thromboembolism,24533,24533,24533,24533,24533,24533
Viral Hepatitis,8688,8688,8688,8688,8688,8688


########## Etapa diferenciada da versão ##########

Não serão excluídos os registros com CID de doenças não evitáveis

In [ ]:
# # Manter na base só dados de CID evitáveis
# dobr_evit_merged_temp02 = dobr_evit_merged_temp01[dobr_evit_merged_temp01['classificacao']=='Avoidable mortality']

# # mostrar dados gerais da tabela após transformações
# tab = dobr_evit_merged_temp02
# print(f'Registro da tabela : {tab.shape[0]}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

#### Resultados do merge de cid_avoid e dobr

##########       Etapa diferenciada da versão      ##########

Atenção para etapa de salvar arquivo, para não sobrescrever outra versão

In [8]:
# Salvar arquivo final de DOBR

dobr_evit_merged_temp01.to_parquet('files_temp/ds5_dobr_evit_merged.parquet')

In [9]:
# Contador

shape_origin = dobr_evit_merged_temp.CAUSABAS.count()
shape_filt = dobr_evit_merged_temp01.CAUSABAS.count()
dif = shape_origin - shape_filt


print_y(f" O número de óbitos após a limpeza e que constituía a base de dados de DO era de {shape_origin} óbitos ")
print_y(f" O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de {dif} óbitos ")
print_y(f" O número de óbitos por CID EVITÁVEIS é de {shape_filt} óbitos ")

 ==>  O número de óbitos após a limpeza e que constituía a base de dados de DO era de 4427475 óbitos 
 ==>  O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de 0 óbitos 
 ==>  O número de óbitos por CID EVITÁVEIS é de 4427475 óbitos 


### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [10]:
# Verificar colunas de população no arquivo censo_pop_clean
censo_temp = pd.read_parquet('files_in_geral/censo_uf.parquet')

# mostrar dados gerais da tabela após transformações
tab = censo_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 540
+----+-------+---------+----------+--------+---------------+-------------+---------------+
|    | Ano   | uf_co   | uf_no    | Sexo   | Cor ou raça   |   pop_total |   pop_under74 |
|----+-------+---------+----------+--------+---------------+-------------+---------------|
|  0 | _2010 | _11     | Rondônia | Homens | Branca        |      274670 |        269314 |
|  1 | _2010 | _11     | Rondônia | Homens | Preta         |       60563 |         59252 |
|  2 | _2010 | _11     | Rondônia | Homens | Amarela       |       10126 |          9986 |
|  3 | _2010 | _11     | Rondônia | Homens | Parda         |      443524 |        437401 |
|  4 | _2010 | _11     | Rondônia | Homens | Indígena      |        6143 |          6037 |
+----+-------+---------+----------+--------+---------------+-------------+---------------+


In [37]:
# Transformar campos de cor e sexo em código
censo_temp01 = censo_temp.copy()

# Transformar valores de sexo em código (utilizado pelo SIM)
censo_temp01['SEXO'] = censo_temp01['Sexo'].map({'Homens':'_1',
                                                    'Mulheres':'_2'})
# Transformar valores nan em 0
censo_temp01['RACACOR'] = censo_temp01['Cor ou raça'].fillna('_0')

# Transformar valores de racacor em código (utilizado pelo SIM)
censo_temp01['RACACOR'] = censo_temp01['RACACOR'].map({'Branca':'_1',
                                                        'Preta':'_2',
                                                        'Amarela':'_3',
                                                        'Parda':'_4',
                                                        'Indígena':'_5',
                                                        'Sem declaração':'_9'})



# Exibir resultados
print(f'Código de sexo: {censo_temp01['SEXO'].unique()}')
print(f'Código de racacor: {censo_temp01['RACACOR'].unique()}')
print(f'Código de ANO: {censo_temp01['Ano'].unique()}')

Código de sexo: ['_1' '_2']
Código de racacor: ['_1' '_2' '_3' '_4' '_5']
Código de ANO: ['_2010' '_2022']


In [38]:

# mostrar dados gerais da tabela após transformações
tab = censo_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 540
+----+-------+---------+----------+--------+---------------+-------------+---------------+--------+-----------+
|    | Ano   | uf_co   | uf_no    | Sexo   | Cor ou raça   |   pop_total |   pop_under74 | SEXO   | RACACOR   |
|----+-------+---------+----------+--------+---------------+-------------+---------------+--------+-----------|
|  0 | _2010 | _11     | Rondônia | Homens | Branca        |      274670 |        269314 | _1     | _1        |
|  1 | _2010 | _11     | Rondônia | Homens | Preta         |       60563 |         59252 | _1     | _2        |
|  2 | _2010 | _11     | Rondônia | Homens | Amarela       |       10126 |          9986 | _1     | _3        |
|  3 | _2010 | _11     | Rondônia | Homens | Parda         |      443524 |        437401 | _1     | _4        |
|  4 | _2010 | _11     | Rondônia | Homens | Indígena      |        6143 |          6037 | _1     | _5        |
+----+-------+---------+----------+--------+---------------+-------------+-----

In [ ]:
# # Criar um campo string que concatene ANO, CODMUN_SIM, Sexo e cor
# censo_pop_mun_4merge_temp02 = censo_pop_mun_4merge_temp01.copy()

# censo_pop_mun_4merge_temp02['POP_MERGE'] = '_' + censo_pop_mun_4merge_temp02['Ano'].astype(str) + censo_pop_mun_4merge_temp02['uf_co'] + censo_pop_mun_4merge_temp02['sexo'] + censo_pop_mun_4merge_temp02['racacor']


# # mostrar dados gerais da tabela após transformações
# tab = censo_pop_mun_4merge_temp02
# print(f'Registro da tabela : {tab.shape[0]}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

In [ ]:
# # Contar quantos valores são repetidos na coluna 'POP_MERGE'
# repeated_values = censo_pop_mun_4merge_temp01['POP_MERGE'].value_counts()

# # Filtrar os valores que aparecem mais de uma vez
# repeated_more_than_once = repeated_values[repeated_values > 1]

# # Exibir os valores repetidos
# print(repeated_more_than_once)

In [16]:
# Exibir arquivo dobr_clean
dobr_merged_temp = pd.read_parquet('files_temp/ds5_dobr_evit_merged.parquet')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4427475
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Intestinal diseases |
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+


##########       Etapa diferenciada da versão      ##########

Como só temos dados de população de Censo de 2010 e 2022, esta etapa do código aplicamos a todos os anos que não são de 2010 um ano proxy de 2023 (serão carregados dados populacionais do Censo de 2010 para o ano de 2010 e para os demais anos, Censo de 2022).

In [20]:
# # Gerar um campo de ano para proxy de 2022 (para ser aplicados aos anos de 19 a 22)
dobr_merged_temp01 = dobr_merged_temp.copy()

dobr_merged_temp01['ANO_PROXY'] = dobr_merged_temp01['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')

# # # Criar um campo string que concatene ANO, CODMUNRES
# dobr_merged_temp01['POP_MERGE'] = dobr_merged_temp01['ANO_PROXY'].astype(str) + dobr_merged_temp['CODMUNRES']

# # mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {dobr_merged_temp01.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4427475
Total de óbitos : 4446922
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+-------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       | ANO_PROXY   |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------+-------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases | _2022       |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Intestinal diseases |

In [21]:
# # Verificar operação acima
print(f'Código de ANO: {dobr_merged_temp01['ANO_OBITO'].unique()}')
print(f'Código de ANO_PROXY: {dobr_merged_temp01['ANO_PROXY'].unique()}')
print(f'Total de óbitos : {dobr_merged_temp01.OBITO_SOMA.sum()}')

Código de ANO: ['_2019' '_2020' '_2021' '_2022' '_2010']
Código de ANO_PROXY: ['_2022' '_2010']
Total de óbitos : 4446922


In [43]:
# # Contar o número de caracteres de cada valor na coluna 'POP_MERGE'
# lengths = dobr_merged_temp01['POP_MERGE'].str.len()
# num_records = lengths[lengths > 12].count()
# print(f"Número de registros de dobr com 'POP_MERGE' maior que 12 caracteres: {num_records}")
# lengths2 = censo_pop_mun_4merge_temp01['POP_MERGE'].str.len()
# num_records2 = lengths2[lengths2 > 12].count()
# print(f"Número de registros de censo_pop_mun com 'POP_MERGE' maior que 12 caracteres: {num_records2}")

Número de registros de dobr com 'POP_MERGE' maior que 12 caracteres: 0
Número de registros de censo_pop_mun com 'POP_MERGE' maior que 12 caracteres: 0


##########       Etapa diferenciada da versão      ##########

Iremos excluir o CAUSABAS, mas agrupar por UF, ano e cause_deaths.
O contador de mortes portanto está atrelado ao cause_deaths.

Atenção também porque, como nesta versão não trabalhamos com município, iremos agregar os dados por UF

In [22]:
dobr_merged_temp01.columns

Index(['CAUSABAS', 'CODMUNRES', 'SEXO', 'RACACOR', 'OBITO_SOMA', 'ANO_OBITO',
       'causes_deaths', 'ANO_PROXY'],
      dtype='object')

In [23]:
dobr_merged_temp01['uf_co'] = dobr_merged_temp01['CODMUNRES'].str[:3]
# # mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))


Registro da tabela : 4427475
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+-------------+---------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       | ANO_PROXY   | uf_co   |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------+-------------+---------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       | _35     |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       | _32     |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       | _33     |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases | _2022       | _35     |
|  4 | A09        | _355100     |      2 |         1 |       

In [24]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS

col_group= ['uf_co', 'SEXO', 'RACACOR', 'ANO_OBITO', 'causes_deaths', 'ANO_PROXY']
dobr_merged_temp02 = dobr_merged_temp01.groupby(col_group)['CAUSABAS'].count().reset_index()

# Renomear o campo CAUSABAS para CONTADOR
dobr_merged_temp02.rename(columns={'CAUSABAS':'CONTADOR'}, inplace=True)


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp02
print(f'Registros da tabela : {tab.shape[0]}')

print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=30))

Registros da tabela : 56415
Total de óbitos : 4427475
+----+---------+--------+-----------+-------------+--------------------------------+-------------+------------+
|    | uf_co   |   SEXO | RACACOR   | ANO_OBITO   | causes_deaths                  | ANO_PROXY   |   CONTADOR |
|----+---------+--------+-----------+-------------+--------------------------------+-------------+------------|
|  0 | _11     |      0 |           | _2019       | causa não evitável             | _2022       |          1 |
|  1 | _11     |      0 |           | _2020       | Certain conditions originating | _2022       |          1 |
|    |         |        |           |             | in the perinatal period        |             |            |
|  2 | _11     |      0 |           | _2021       | causa não evitável             | _2022       |          1 |
|  3 | _11     |      0 | 1         | _2019       | Cerebrovascular diseases       | _2022       |          1 |
|  4 | _11     |      0 | 1         | _2019       

In [25]:
# Tratamento de códigos de dobr para merge
print(f'Código de sexo: {dobr_merged_temp02['SEXO'].unique()}')
print(f'Código de racacor: {dobr_merged_temp02['RACACOR'].unique()}')

Código de sexo: ['0' '1' '1   ' '2' '2   ' '9   ']
Código de racacor: [' ' '1' '2' '4' '3' '5' '       ' '1      ' '2      ' '3      ' '4      '
 '5      ']


In [26]:
# Transformar valores RACACOR e SEXO
dobr_merged_temp03 = dobr_merged_temp02.copy()

# Transformar valores RACACOR
# dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].replace([' ', ''], '0')
# dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].fillna('0')
dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].astype(str)
dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].str[:1]
dobr_merged_temp03['RACACOR'] = dobr_merged_temp03['RACACOR'].replace(' ','9')
dobr_merged_temp03['RACACOR'] = '_' + dobr_merged_temp03['RACACOR'].astype(str)

# Transformar valores SEXO
dobr_merged_temp03['SEXO'] = dobr_merged_temp03['SEXO'].astype(str)
dobr_merged_temp03['SEXO'] = dobr_merged_temp03['SEXO'].str[:1]
dobr_merged_temp03['SEXO'] = dobr_merged_temp03['SEXO'].replace('0','9')
dobr_merged_temp03['SEXO'] = '_' + dobr_merged_temp03['SEXO'].astype(str)



print(f'Código de RACACOR: {dobr_merged_temp03["RACACOR"].unique()}')
print(f'Código de SEXO: {dobr_merged_temp03["SEXO"].unique()}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')


Código de RACACOR: ['_9' '_1' '_2' '_4' '_3' '_5']
Código de SEXO: ['_9' '_1' '_2']
Total de óbitos : 4427475


In [27]:
# Apresentar total de registros de DOBR sem identificação de SEXO e RACACOR
dobr_merged_temp03_fault = dobr_merged_temp03.loc[(dobr_merged_temp03['SEXO'] == '_9') | (dobr_merged_temp03['RACACOR'] == '_9')]
sexo_fault = dobr_merged_temp03.loc[(dobr_merged_temp03['SEXO']== '_9')]
racacor_fault = dobr_merged_temp03.loc[(dobr_merged_temp03['RACACOR']== '_9')]

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp03_fault
print(f'Registros de RACACOR sem qualidade : {racacor_fault.CONTADOR.sum()}')
print(f'Registros de SEXO sem qualidade : {sexo_fault.CONTADOR.sum()}')
print(f'Registros total sem qualidade : {tab.CONTADOR.sum()}')

faults_perc = (tab.CONTADOR.sum()/dobr_merged_temp03.CONTADOR.sum())*100

print_y(f'Total de óbitos sem dados de qualidade em SEXO e RACACOR: {tab.CONTADOR.sum()}')
print_y(f'Percentual de óbitos sem dados de qualidade em SEXO e RACACOR: {faults_perc}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=30))

Registros de RACACOR sem qualidade : 133486
Registros de SEXO sem qualidade : 1264
Registros total sem qualidade : 134455
 ==> Total de óbitos sem dados de qualidade em SEXO e RACACOR: 134455
 ==> Percentual de óbitos sem dados de qualidade em SEXO e RACACOR: 3.0368325061124004


In [28]:
# Excluir valores sem qualidade da tabela
dobr_merged_temp04 = dobr_merged_temp03[(dobr_merged_temp03['RACACOR'] != '_9') & (dobr_merged_temp03['SEXO'] != '_9')]

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')

print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=30))

Registros da tabela : 47498
Total de óbitos : 4293020
+----+---------+--------+-----------+-------------+--------------------------------+-------------+------------+
|    | uf_co   | SEXO   | RACACOR   | ANO_OBITO   | causes_deaths                  | ANO_PROXY   |   CONTADOR |
|----+---------+--------+-----------+-------------+--------------------------------+-------------+------------|
| 88 | _11     | _1     | _1        | _2019       | Abscess of lung and            | _2022       |          1 |
|    |         |        |           |             | mediastinum pyothorax          |             |            |
| 89 | _11     | _1     | _1        | _2019       | Accidental Injuries            | _2022       |         47 |
| 90 | _11     | _1     | _1        | _2019       | Acute lower respiratory        | _2022       |          1 |
|    |         |        |           |             | infections                     |             |            |
| 91 | _11     | _1     | _1        | _2019       

In [30]:
# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=30))

Registros da tabela : 47498
+----+---------+--------+-----------+-------------+--------------------------------+-------------+------------+
|    | uf_co   | SEXO   | RACACOR   | ANO_OBITO   | causes_deaths                  | ANO_PROXY   |   CONTADOR |
|----+---------+--------+-----------+-------------+--------------------------------+-------------+------------|
| 88 | _11     | _1     | _1        | _2019       | Abscess of lung and            | _2022       |          1 |
|    |         |        |           |             | mediastinum pyothorax          |             |            |
| 89 | _11     | _1     | _1        | _2019       | Accidental Injuries            | _2022       |         47 |
| 90 | _11     | _1     | _1        | _2019       | Acute lower respiratory        | _2022       |          1 |
|    |         |        |           |             | infections                     |             |            |
| 91 | _11     | _1     | _1        | _2019       | Acute pancreatitis      

In [39]:
censo_temp01.columns

Index(['Ano', 'uf_co', 'uf_no', 'Sexo', 'Cor ou raça', 'pop_total',
       'pop_under74', 'SEXO', 'RACACOR'],
      dtype='object')

In [40]:
# Definir campos de censo_pop para merge
# censo_pop_mun_4merge_temp02 = censo_pop_mun_4merge_temp01[['ano_cod',
#                                                            'uf_co',
#                                                            'sexo_cod',
#                                                            'Sexo',
#                                                            'racacor_cod',
#                                                            'Cor ou Raça',
#                                                            'pop_under74','UF','mun_no']]

# Realizar o merge dos dataframes
dobr_merged_temp05 = pd.merge(dobr_merged_temp04, censo_temp01, 
                              left_on=['ANO_PROXY','uf_co','SEXO','RACACOR'],
                              right_on=['Ano','uf_co','SEXO','RACACOR'],
                              how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp05
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
tab.head()

Registros da tabela : 47498
Total de óbitos : 4293020


,uf_co,SEXO,RACACOR,ANO_OBITO,causes_deaths,ANO_PROXY,CONTADOR,Ano,uf_no,Sexo,Cor ou raça,pop_total,pop_under74
0,_11,_1,_1,_2019,Abscess of lung and mediastinum pyothorax,_2022,1,_2022,Rondônia,Homens,Branca,236680,228640
1,_11,_1,_1,_2019,Accidental Injuries,_2022,47,_2022,Rondônia,Homens,Branca,236680,228640
2,_11,_1,_1,_2019,Acute lower respiratory infections,_2022,1,_2022,Rondônia,Homens,Branca,236680,228640
3,_11,_1,_1,_2019,Acute pancreatitis,_2022,3,_2022,Rondônia,Homens,Branca,236680,228640
4,_11,_1,_1,_2019,Alcohol-specific disorders and poisonings,_2022,12,_2022,Rondônia,Homens,Branca,236680,228640


In [41]:
# Contar quantos erros do merge
merged_faults = dobr_merged_temp05['pop_under74'].isna().sum()
merged_faults

0

In [42]:
# Ajustar dados de ano
dobr_merged_temp05['ANO_OBITO'] = dobr_merged_temp05['ANO_OBITO'].str[1:5]

In [44]:
# Filtra colunas
column_order = [
                #  'uf_co',
                #  'SEXO',
                #  'RACACOR',
                 'ANO_OBITO',
                #  'ANO_PROXY',
                # 'Ano',
                'uf_no',
                'causes_deaths',
                'Sexo',
                'Cor ou raça',
                #  'pop_total',
                'pop_under74',
                'CONTADOR',
                #  'sexo_cod',
                #  'racacor_cod',
                #  'ano_cod'
                ]

# Reorder the columns in the DataFrame
dobr_merged_temp06 = dobr_merged_temp05[column_order]


# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp06
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql', maxcolwidths=50))


Registros da tabela : 47498
Total de óbitos : 4293020
+----+-------------+----------+-------------------------------------------+--------+---------------+---------------+------------+
|    |   ANO_OBITO | uf_no    | causes_deaths                             | Sexo   | Cor ou raça   |   pop_under74 |   CONTADOR |
|----+-------------+----------+-------------------------------------------+--------+---------------+---------------+------------|
|  0 |        2019 | Rondônia | Abscess of lung and mediastinum pyothorax | Homens | Branca        |        228640 |          1 |
|  1 |        2019 | Rondônia | Accidental Injuries                       | Homens | Branca        |        228640 |         47 |
|  2 |        2019 | Rondônia | Acute lower respiratory infections        | Homens | Branca        |        228640 |          1 |
|  3 |        2019 | Rondônia | Acute pancreatitis                        | Homens | Branca        |        228640 |          3 |
|  4 |        2019 | Rondônia | Alco

##### Arquivo tratado final

##########       Etapa diferenciada da versão      ##########

Atenção na hora de salvar o arquivo temporário com versão, formato e local

In [45]:
dobr_merged_temp06.to_parquet('files_temp/ds5_dobr_merged.parquet')

In [46]:
# Resultados

obito_origin = dobr_merged_temp.CAUSABAS.count()
obito_final = dobr_merged_temp06.CONTADOR.sum()
dif = obito_origin - obito_final


print_y(f" O número de óbitos após merge com tabela de CIDs evitáveis era de {obito_origin} óbitos ")
print_y(f" Foram excluídos, por apresentar problemas para carga de informações de município, {dif} óbitos ")
print_y(f" O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de {obito_final} óbitos ")


# # Filtrar registros onde pop_under74 é null
# dobr_merged_pop_nulos = dobr_merged_pop[dobr_merged_pop['pop_under74'].isnull()]

# # Exibir os registros filtrados
# dobr_merged_pop_nulos['obito_count'].sum()

 ==>  O número de óbitos após merge com tabela de CIDs evitáveis era de 4427475 óbitos 
 ==>  Foram excluídos, por apresentar problemas para carga de informações de município, 134455 óbitos 
 ==>  O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de 4293020 óbitos 


# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 4 - BR/CIDs/uf/Mun


O primeiro dataset que iremos construir tem as seguintes características:

* set = Brasil
* indicador = causes_deaths
* dimen = UF
* subgroup = município
* estimate = mortalidade por ano, causa (indicador) e subgroup
* set-avg = mortalidade por ano e causa (indicador)

Estas escolhas implicam em podermos disponibilizar ao usuário do HEAT a possibilidade de ver os municípios (dimensões) de maneira agregada por UF. 


In [3]:
# Define the URL and the destination path
url = "https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm"
destination_path = "downloads_outros/heat_plus_template_validation.xlsm"

# Download the file
response = requests.get(url)
with open(destination_path, 'wb') as file:
    file.write(response.content)

# Note: Enabling macros programmatically is not recommended due to security risks.
# Macros should be enabled manually in Excel settings.

In [47]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files_temp/ds5_dobr_merged.parquet')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 47498
Total de óbitos : 4293020
+----+-------------+----------+-------------------------------------------+--------+---------------+---------------+------------+
|    |   ANO_OBITO | uf_no    | causes_deaths                             | Sexo   | Cor ou raça   |   pop_under74 |   CONTADOR |
|----+-------------+----------+-------------------------------------------+--------+---------------+---------------+------------|
|  0 |        2019 | Rondônia | Abscess of lung and mediastinum pyothorax | Homens | Branca        |        228640 |          1 |
|  1 |        2019 | Rondônia | Accidental Injuries                       | Homens | Branca        |        228640 |         47 |
|  2 |        2019 | Rondônia | Acute lower respiratory infections        | Homens | Branca        |        228640 |          1 |
|  3 |        2019 | Rondônia | Acute pancreatitis                        | Homens | Branca        |        228640 |          3 |
|  4 |        2019 | Rondônia | Alco

In [38]:
# # Agregar os dados sem CID e gerar colunas de soma de obito_count e pop_under74
# heat_semCID = heat_.groupby(['UF', 'mun_nome', 'CODMUNRES', 'SEXO', 'RACACOR', 'ANO']).agg({
#     'obito_count': 'sum',
#     'pop_under74': 'sum'
# }).reset_index()

##########       Etapa diferenciada da versão      ##########

Para melhor registro, utilizamos o label mort_ind_uf para deixar claro que o indicador é relativo a UF e causa de morte

In [48]:
# Gerar o indicador de mortalidade (por 100 mil habitantes) e exibir as primeiras linhas do DataFrame
heatds_temp01 = heatds_temp.copy()

heatds_temp01['mort_ind_uf'] = (heatds_temp01['CONTADOR']/heatds_temp01['pop_under74'])*100000

# mostrar dados gerais da tabela após transformações
tab = heatds_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql',maxcolwidths=20))

Registros da tabela : 47498
Total de óbitos : 4293020
+----+-------------+----------+---------------------+--------+---------------+---------------+------------+---------------+
|    |   ANO_OBITO | uf_no    | causes_deaths       | Sexo   | Cor ou raça   |   pop_under74 |   CONTADOR |   mort_ind_uf |
|----+-------------+----------+---------------------+--------+---------------+---------------+------------+---------------|
|  0 |        2019 | Rondônia | Abscess of lung and | Homens | Branca        |        228640 |          1 |      0.437369 |
|    |             |          | mediastinum         |        |               |               |            |               |
|    |             |          | pyothorax           |        |               |               |            |               |
|  1 |        2019 | Rondônia | Accidental Injuries | Homens | Branca        |        228640 |         47 |     20.5563   |
|  2 |        2019 | Rondônia | Acute lower         | Homens | Branca        |

In [49]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_uf'])]
inf_count = np.isinf(heatds_temp01['mort_ind_uf']).sum()
nulos_mort_ind_mun = heatds_temp01['mort_ind_uf'].isnull().sum()


# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
print(f"Valores nulos em 'mort_ind_mun': {nulos_mort_ind_mun}")



# Exportar registros de indicador de mortalidade infinitos [inf]
# registros_inf.to_csv('exportar/relatorio_reg_mort_ind_inf.csv')

Número de valores infinitos em mort_ind_mun: 0
Valores nulos em 'mort_ind_mun': 0


In [ ]:
# # Limpar a base de registros de indicador de mortalidade infinitos [inf]
# heat_ds_treat2 = heat_ds_treat.copy()

# # Substituir valores infinitos por NaN
# heat_ds_treat2['mort_ind_mun'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# # Rodar se for necessário excluir valores nulos
# heatds_temp02 = heatds_temp01.copy()

# # Excluir registros com NaN na coluna mort_ind_mun
# heatds_temp02.dropna(subset=['mort_ind_mun'], inplace=True)

# # Mudar o nome da coluna mort_ind para mort_ind_mun
# # heat_ds_treat2.rename(columns={'mort_ind':'mort_ind_mun'}, inplace=True)


# # Verificar as primeiras linhas do DataFrame atualizado
# heat_ds_treat1.head()

In [50]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp01.describe()

,pop_under74,CONTADOR,mort_ind_uf
count,4.749800e+04,47498.000000,47498.000000
mean,1.264342e+06,90.383174,12.773144
std,1.904628e+06,672.551160,54.835053
min,3.390000e+02,1.000000,0.007602
25%,1.697760e+05,2.000000,0.467549
50%,6.289610e+05,6.000000,1.694513
75%,1.445480e+06,27.000000,7.913271
max,1.315425e+07,56500.000000,4424.778761


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [ ]:
# # Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
# dobr_grouped_uf = heat_ds_treat2.groupby(['UF',
#                                              'ANO_OBITO',
#                                             #  'SEXO',
#                                             #  'RACACOR'
#                                              ]).agg({'CONTADOR': 'sum',
#                                                     'pop_under74': 'sum',
#                                                     'mort_ind_mun': 'mean'
#                                                 }).reset_index()
# dobr_grouped_uf.rename(columns={'mort_ind_mun': 'mort_ind_uf',
#                                 'CONTADOR': 'obito_count_uf',
#                                 'pop_under74': 'pop_under74_uf',
#                                 }, inplace=True)


# # Exibir as primeiras linhas do DataFrame resultante
# dobr_grouped_uf.head()

In [ ]:
# dobr_grouped_uf.describe()

In [ ]:
# Salvar arquivo por UF para exportação
# dobr_grouped_uf.to_excel('exportar/dobr_grouped_uf_v2.xlsx', index=False)

In [ ]:
# heat_ds_treat2.columns

In [ ]:
# # Trazer dados por UF (setting e setting_average)

# # Realizar o merge entre heat_ e uf_sett
# heat_treat00 = pd.merge(heat_ds_treat2, dobr_grouped_uf, on=['UF',
#                                                                 'ANO_OBITO',
#                                                                 # 'SEXO',
#                                                                 # 'RACACOR'
#                                                                 ], how='left')

# # Exibir as primeiras linhas do dataframe resultante
# heat_treat00.head()

In [ ]:
# # Teste de aplicação de setting_average (médias por UF)
# filtered_data = heat_treat00[   (heat_treat00['ANO_OBITO'] == '_2022') &
#                                 (heat_treat00['UF'] == 'AC') 
#                                 # & (heat_treat00['SEXO'] == '_1') &
#                                 #(heat_treat00['RACACOR'] == '_4')
#                               ]
# filtered_data.head()

In [ ]:
# heat_treat00.columns

In [ ]:
# heat_treat00.pop_under74_uf.sum()

In [ ]:
# obito_num = heat_treat00.CONTADOR.sum()
# pop_under74 = heat_treat00.pop_under74.sum()

# print(f"\033[33m ==> Número de mortes: {obito_num} \033[0m")
# print(f"\033[33m ==> Número de população abaixo de 75 anos é de {pop_under74} \033[0m")  


In [ ]:
# heat_template = row
# heat_template

##### Criar colunas do dataset

##########       Etapa diferenciada da versão      ##########

Esta etapa precisa bastante atenção.

Se considerarmos causes_deaths como dimensão, ela precisa ser ignorada na criação so set-avg.

Pelo contrário, se formos tratar causes_deaths como indicador, ele precisa ser considerado na criação do set-avg.

Vamos realizar abaixo causes_deaths como indicador.

In [51]:
heatds_temp01.columns

Index(['ANO_OBITO', 'uf_no', 'causes_deaths', 'Sexo', 'Cor ou raça',
       'pop_under74', 'CONTADOR', 'mort_ind_uf'],
      dtype='object')

In [53]:
# Na média ponderada de mortalidade evitável, pegamos os valores de mortalidade para cada município e dividimos pela população, para conseguir o valor para o Brasil e por ano

# Função para calcular a média ponderada
def weighted_avg(group):
    return np.average(group['mort_ind_uf'], weights=group['pop_under74'])

# Agrupar os dados pelo ano e aplicar a função de média ponderada
weighted_avg_per_year = heatds_temp01.groupby(['ANO_OBITO','causes_deaths']).apply(weighted_avg).reset_index()

# Renomear as colunas para melhor entendimento
weighted_avg_per_year.columns = ['ANO_OBITO','causes_deaths','set_avg_BR']

# Exibir o resultado
print(weighted_avg_per_year)

    ANO_OBITO                              causes_deaths  set_avg_BR
0        2010                           Abdominal hernia    0.435484
1        2010  Abscess of lung and mediastinum pyothorax    0.303694
2        2010                        Accidental Injuries    9.582480
3        2010         Acute lower respiratory infections    0.228161
4        2010                         Acute pancreatitis    1.178403
..        ...                                        ...         ...
393      2022               Upper respiratory infections    0.306096
394      2022                                  Varicella    0.052776
395      2022                     Venous thromboembolism    3.074914
396      2022                            Viral Hepatitis    0.718961
397      2022                         causa não evitável  171.615930

[398 rows x 3 columns]


/tmp/ipykernel_2983/606829036.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_per_year = heatds_temp01.groupby(['ANO_OBITO','causes_deaths']).apply(weighted_avg).reset_index()


In [54]:
# Realizar o merge entre heat_ds_treat2 e grouped_data usando a coluna ANO_OBITO
heatds_temp02 = pd.merge(heatds_temp01, weighted_avg_per_year, on=['ANO_OBITO','causes_deaths'], how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
tab.head()
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 47498
Total de óbitos : 4293020


,ANO_OBITO,uf_no,causes_deaths,Sexo,Cor ou raça,pop_under74,CONTADOR,mort_ind_uf,set_avg_BR
0,2019,Rondônia,Abscess of lung and mediastinum pyothorax,Homens,Branca,228640,1,0.437369,0.319505
1,2019,Rondônia,Accidental Injuries,Homens,Branca,228640,47,20.556333,9.804835
2,2019,Rondônia,Acute lower respiratory infections,Homens,Branca,228640,1,0.437369,0.337507
3,2019,Rondônia,Acute pancreatitis,Homens,Branca,228640,3,1.312106,1.215954
4,2019,Rondônia,Alcohol-specific disorders and poisonings,Homens,Branca,228640,12,5.248425,8.368542


##########       Etapa diferenciada da versão      ##########

Para esta versão
* set = Brasil
* indicador = causes_deaths
* dimen = UF
* subgroup = Sexo & Cor ou raça
* estimate = mortalidade por ano, causa (indicador) e subgroup
* set-avg = mortalidade por ano e causa (indicador)

In [55]:
# Etapas para gerar indicator_abbr exclusivos

# Etapa 1 - Gerar uma tabela apenas com valores exclusivos de heatds_temp02.causes_deaths, em ordem crescente
unique_causes_deaths = heatds_temp02[['causes_deaths']].drop_duplicates().sort_values(by='causes_deaths').reset_index(drop=True)

# Etapa 2 - Criar coluna indicator_abbr com os primeiros 5 caracteres de causes_deaths
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths['causes_deaths'].str[:5]

# Etapa 3 - Para aqueles indicator_abbr que tiverem o mesmo nome, gerar sufixo incremental a partir de 1
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths.groupby('indicator_abbr').cumcount().astype(str).radd(unique_causes_deaths['indicator_abbr'])

# Etapa 3.1 - excluir valores que terminam com 0
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths['indicator_abbr'].str.replace('0', '')

# Exibir a tabela resultante
unique_causes_deaths

,causes_deaths,indicator_abbr
0,Abdominal hernia,Abdom
1,Abscess of lung and mediastinum pyothorax,Absce
2,Accidental Injuries,Accid
3,Acute lower respiratory infections,Acute
4,Acute pancreatitis,Acute1
...,...,...
76,Varicella,Varic
77,Venous thromboembolism,Venou
78,Viral Hepatitis,Viral
79,Whooping cough,Whoop


In [56]:
# Fazer merge com informações de indicator_abbr exclusivos
heatds_temp03 = pd.merge(heatds_temp02,unique_causes_deaths,on='causes_deaths', how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 47498
Total de óbitos : 4293020


,ANO_OBITO,uf_no,causes_deaths,Sexo,Cor ou raça,pop_under74,CONTADOR,mort_ind_uf,set_avg_BR,indicator_abbr
0,2019,Rondônia,Abscess of lung and mediastinum pyothorax,Homens,Branca,228640,1,0.437369,0.319505,Absce
1,2019,Rondônia,Accidental Injuries,Homens,Branca,228640,47,20.556333,9.804835,Accid
2,2019,Rondônia,Acute lower respiratory infections,Homens,Branca,228640,1,0.437369,0.337507,Acute
3,2019,Rondônia,Acute pancreatitis,Homens,Branca,228640,3,1.312106,1.215954,Acute1
4,2019,Rondônia,Alcohol-specific disorders and poisonings,Homens,Branca,228640,12,5.248425,8.368542,Alcoh


In [59]:
# Gerar colunas sem informação ou com informação padronizada

# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heatds_temp04 = heatds_temp03.copy() # caso seja agrupado por UF anteriormente

heatds_temp04['setting'] = 'Brasil'
# heat_treat01['setting_average'] = set_avg_BR

# heatds_temp03['indicator_abbr'] = 'AVM'
# heatds_temp03['indicator_abbr'] = heatds_temp03['causes_deaths'].str.slice(0, 5)
# heatds_temp03['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heatds_temp04['indicator_name'] = heatds_temp04['causes_deaths'] + '(deaths per 100 000 population)'

heatds_temp04['subgroup'] = heatds_temp04['Sexo'] + ' - cor/raça ' + heatds_temp04['Cor ou raça']
heatds_temp04['source'] = 'SIM/SUS & Censo/IBGE'
heatds_temp04['favourable_indicator'] = 0
heatds_temp04['indicator_scale'] = 100000
heatds_temp04['ordered_dimension'] = 0
heatds_temp04['subgroup_order'] = 0
heatds_temp04['reference_subgroup'] = 0
heatds_temp04['note'] = ''
heatds_temp04['se'] = ''
heatds_temp04['ci_lb'] = ''
heatds_temp04['ci_ub'] = ''
heatds_temp04['iso3'] = 'BRA'

# Renomear colunas de heat_treat01
heatds_temp04 = heatds_temp04.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'uf_no':'dimension',
    # 'mun_no':'subgroup',
    # 'mort_ind_mun':'estimate',
    'mort_ind_uf':'estimate',
    'set_avg_BR':'setting_average',
    # 'mort_ind_uf':'setting_average'
})

# Ajustar coluna de ano
heatds_temp04['date'] = heatds_temp04['date'].str.replace('_','').astype(int)


# mostrar dados gerais da tabela após transformações
tab = heatds_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()


Registros da tabela : 47498
Total de óbitos : 4293020


,date,dimension,causes_deaths,Sexo,Cor ou raça,population,CONTADOR,estimate,setting_average,indicator_abbr,...,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3
0,2019,Rondônia,Abscess of lung and mediastinum pyothorax,Homens,Branca,228640,1,0.437369,0.319505,Absce,...,0,100000,0,0,0,,,,,BRA
1,2019,Rondônia,Accidental Injuries,Homens,Branca,228640,47,20.556333,9.804835,Accid,...,0,100000,0,0,0,,,,,BRA
2,2019,Rondônia,Acute lower respiratory infections,Homens,Branca,228640,1,0.437369,0.337507,Acute,...,0,100000,0,0,0,,,,,BRA
3,2019,Rondônia,Acute pancreatitis,Homens,Branca,228640,3,1.312106,1.215954,Acute1,...,0,100000,0,0,0,,,,,BRA
4,2019,Rondônia,Alcohol-specific disorders and poisonings,Homens,Branca,228640,12,5.248425,8.368542,Alcoh,...,0,100000,0,0,0,,,,,BRA


In [285]:
# # Trazer labels para sexo e raçacor
# heat_treat02 = heat_treat01.copy()

# heat_treat02['SEXO'] = heat_treat02['SEXO'].map({
#                                             '_1':'Homens',
#                                             '_2':'Mulheres'
#                                             })
# heat_treat02['RACACOR'] = heat_treat02['RACACOR'].map({
#                                             '_1':'Branca',
#                                             '_2':'Preta',
#                                             '_3':'Amarela',
#                                             '_4':'Parda',
#                                             '_5':'Indígena',
#                                             '_9':'Sem declaração'
#                                             })

# heat_treat02['subgroup'] = heat_treat02['SEXO'] + ' - raça/cor '  +  heat_treat02['RACACOR']


In [ ]:
# heat_treat02['subgroup'].unique()

In [ ]:
# heat_treat02.columns

In [60]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  


# Reorder the columns in the DataFrame
heatds_temp05 = heatds_temp04[column_order]


In [61]:
# Transformar estimate e setting_average em int
heatds_temp06 = heatds_temp05.copy()

heatds_temp06['estimate'] = heatds_temp06['estimate'].astype(int)
heatds_temp06['setting_average'] = heatds_temp06['setting_average'].astype(int)

# mostrar dados gerais da tabela após transformações
tab = heatds_temp06
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 47498


,setting,date,source,indicator_abbr,indicator_name,dimension,subgroup,estimate,se,ci_lb,ci_ub,population,note,setting_average,iso3,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup
0,Brasil,2019,SIM/SUS & Censo/IBGE,Absce,Abscess of lung and mediastinum pyothorax(deat...,Rondônia,Homens - cor/raça Branca,0,,,,228640,,0,BRA,0,100000,0,0,0
1,Brasil,2019,SIM/SUS & Censo/IBGE,Accid,Accidental Injuries(deaths per 100 000 populat...,Rondônia,Homens - cor/raça Branca,20,,,,228640,,9,BRA,0,100000,0,0,0
2,Brasil,2019,SIM/SUS & Censo/IBGE,Acute,Acute lower respiratory infections(deaths per ...,Rondônia,Homens - cor/raça Branca,0,,,,228640,,0,BRA,0,100000,0,0,0
3,Brasil,2019,SIM/SUS & Censo/IBGE,Acute1,Acute pancreatitis(deaths per 100 000 population),Rondônia,Homens - cor/raça Branca,1,,,,228640,,1,BRA,0,100000,0,0,0
4,Brasil,2019,SIM/SUS & Censo/IBGE,Alcoh,Alcohol-specific disorders and poisonings(deat...,Rondônia,Homens - cor/raça Branca,5,,,,228640,,8,BRA,0,100000,0,0,0


##########       Etapa diferenciada da versão      ##########

Atenção da hora de salvar o arquivo, com as informações corretas de versão

In [62]:
### Arquivo pronto para exportação
heatds_temp06.to_excel('files_out/ds5_BR_CID_UF_sexo&cor.xlsx', index=False)

In [63]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('files_in_geral/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('files_out/ds5_BR_CID_UF_sexo&cor.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('files_out/ds5_BR_CID_UF_sexo&cor.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados